In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# Scikit-learn Workflow
 - Understand how to write custom classes in scikit-learn
 - Pipelines
 - Feature Unions

## Writing custom estimators and transformers

The scikit-learn library has a wealth of functionality available in its [classes](http://scikit-learn.org/stable/modules/classes.html). Occasionally you might want to customize the behavior of these classes, for example to add in functionality or for engineering reasons.

Scikit-learn has some detailed documentation on customizing these classes - start [here](http://scikit-learn.org/stable/developers/contributing.html#apis-of-scikit-learn-objects) and make sure to get to the "Rolling your own Estimator" section. 

This notebook contains a bit more exposition and some examples for implementing custom classes. Think of it as a field guide to writing your own pluggable functionality.

All estimators (e.g. linear regression, kmeans, etc ...) support `fit` and `predict` methods.  In fact, you can build your own by inheriting from classes in `sklearn.base` by using this template:                                                                                                 
``` python                                                                                                                                        
class Estimator(base.BaseEstimator, base.RegressorMixin):
  def __init__(self, ...):
    # initialization code
  
  def fit(self, X, y):
    # fit the model ...
    return self
    
  def predict(self, X):
    return # prediction
    
  def score(self, X, y):
    return # custom score implementation
```

Conforming to this convention has the benefit that many tools (e.g. cross-validation, grid search) rely on this interface so you can use your new estimators with the existing `sklearn` infrastructure.                                                                 
                                                                                
For example `model_selection.GridSearchCV` takes an estimator and some hyperparameters as arguments, and returns another estimator.  Upon fitting, it fits the best model (based on the inputted hyperparameters) and uses that for prediction.                                                                    
                                                                                
Of course, we sometimes need to process or transform the data before we can do machine-learning on it.  `sklearn` has Transformers to help with this.  They implement this interface:
``` python
class Transformer(base.BaseEstimator, base.TransformerMixin):
  def __init__(self, ...):
    # initialization code
    
  def fit(self, X, y=None):
    # fit the transformation
    return self
  
  def transform(self, X):
    return ... # transformation
```

A comprehensive `.fit_transform` is implemented based on the `.fit` and `.transform` methods in `base.TransformerMixin` ([docs](http://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html)). However, especially for transformers, `.fit` is often empty and only `.transform` actually does something.

The following is some example code to demonstrate the usage of custom classes.

In [ ]:
import numpy as np
import scipy as sp
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2, random_state=42)
boston.data.shape, X_train.shape, X_test.shape

In [ ]:
class CenteringTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Centers the features about 0
    """
    def fit(self, X, y=None):
        self.means = X.mean(axis=0)
        return self
    
    def transform(self, X):
        return X - self.means

In [ ]:
ct = CenteringTransformer()
ct.fit(X_train)
#Note: trained on train data, testing on test data
X_test.mean(), ct.transform(X_test).mean()

Some estimators and transformers take arguments.  These can be specified in the `__init__` function.

In [ ]:
class TruncateTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Returns the first k columns of a feature array
    """
    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        return self

    def transform(self, X):
        return X[:,:self.k]

In [ ]:
truncator = TruncateTransformer(2)
truncator.transform(X_train).shape

Here's a silly estimator that returns a values based on whether there are more positive or negative features for an observation.

In [ ]:
class PosNegEstimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    """
    Predict mean values based on whether there are more positive or negative features.
    
    Yes, this will do a terrible job.
    """
    def __init__(self):
        self.pos_mean = 0
        self.neg_mean = 0
    
    def fit(self, X, y):
        pos_rows = (X > 0).sum(axis=1) > X.shape[1]/2
        self.pos_mean = y[pos_rows].mean() if sum(pos_rows) > 0 else 0
        self.neg_mean = y[~pos_rows].mean() if sum(~pos_rows) > 0 else 0
        return self
    
    def predict(self, X):
        pos_rows = (X > 0).sum(axis=1) > X.shape[1]/2
        y = np.zeros(X.shape[0])
        y[pos_rows] = self.pos_mean
        y[~pos_rows] = self.neg_mean
        return y

In [ ]:
pne = PosNegEstimator()
pne.fit(X_train, y_train)
pne.predict(X_test)
pne.score(X_test, y_test)

The RegressorMixin provides a `.score` method that calculates the coefficient of determination.  In this case, all rows are positive, so this is just a mean model.

In [ ]:
ct = CenteringTransformer()
pne = PosNegEstimator()

ct.fit(X_train, y_train)
X_train_center = ct.transform(X_train)
pne.fit(X_train_center, y_train)
X_test_center = ct.transform(X_test)
pne.score(X_test_center, y_test)

The fit method returns the `self` object, so that method calls can be chained.  The `.fit_transform` method is equivalent to chaining the `.transform` method to the `.fit` method.  Together, these can turn the above into a one-liner.

In [ ]:
pne.fit(ct.fit_transform(X_train), y_train).score(ct.transform(X_test), y_test)

Connecting transformers to estimators can become tedious.  One way around this is to build an estimator that combines them into one object.

In [ ]:
class ShellEstimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    """
    A shell estimator that combines a transformer and regressor into a single object.
    """
    def __init__(self, transformer, model):
        self.transformer = transformer
        self.model = model

    def fit(self, X, y):
        X_trans = self.transformer.fit(X, y).transform(X)
        self.model.fit(X_trans, y)
        return self
    
    def score(self, X, y):
        X_test = self.transformer.transform(X)
        return self.model.score(X_test, y)

    def predict(self, X):
        X_test = self.transformer.transform(X)
        return self.model.predict(X_test)

In [ ]:
truncator = TruncateTransformer(2)
X_train_k = truncator.transform(X_train)
X_test_k = truncator.transform(X_test)

linreg = LinearRegression()
k_model = linreg.fit(X_train_k, y_train)
y_pred_k = k_model.predict(X_test_k)
print k_model.score(X_test_k, y_test)

In [ ]:
k_shell = ShellEstimator(TruncateTransformer(2), LinearRegression(fit_intercept=True))
k_shell.fit(X_train, y_train)
assert((k_shell.predict(X_test) == y_pred_k).all())
print k_shell.score(X_test, y_test)

## Pipelines

It turns out there's a built-in tool to chain together our transformers and estimators into one unit, and it scales much easier than custom estimators. They're called pipelines. The following code would replace all the fitting and scoring code above.  That is, the pipeline itself is an estimator (and implements the `.fit` and `.predict` methods).  Note that a pipeline can have multiple transformers chained up but at most one (optional) terminal estimator.

In [ ]:
from sklearn import pipeline

k_pipe = pipeline.Pipeline([
  ('truncate', TruncateTransformer(2)),
  ('linreg', LinearRegression(fit_intercept=True))
  ])
k_pipe.fit(X_train, y_train)
print k_pipe.score(X_test, y_test)

The hyper-parameters of the compenent elements are available as parameters in the pipeline with the `element__param_name` format.  This means the pipelines can be used in cross-validation routines.

In [ ]:
k_pipe.get_params()

## Feature Unions

[Feature unions](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html) are designed to get around the problem that you might not be able to prepare your desired feature matrix with just one series of transformations. For example, you might have text features, categorical features, and a couple different kinds of numerical features and want to feed them all into the same estimator or pipeline. Each feature type would require a different kind of transformer.

What the feature union does is a kind of *parallel* transformation operation using multiple transformers and consolidating them into one output matrix - which can then be fed into an estimator or pipeline. You can imagine that between the serial behavior of pipelines and the parallel behavior of feature unions you can create complex multi-stage workflows.

This example code applies several different transformations to X before throwing the features into a Linear Regressor.

In [ ]:
class ReverseTruncateTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Returns the last k columns of a feature array
    """
    def __init__(self, k):
        self.k = k

    def fit(self, X, y):
        return self

    def transform(self, X):
        return X[:,-self.k:]
     
all_features = pipeline.FeatureUnion([
  ('first two cols', TruncateTransformer(2)),
  ('last two cols', ReverseTruncateTransformer(2))
  ])
k_union = pipeline.Pipeline([
  ("features", all_features),
  ("linreg", LinearRegression(fit_intercept=True))
  ])
k_union.fit(X_train, y_train)
print k_union.score(X_test, y_test)

You can also use feature unions to combine the predictions of multiple estimators. If you rewrite your estimators as transformers and feed them into a feature union which has eg. a linear regressor as an estimator, you'll be able to automatically weight and combine each individual prediction into an ensemble model.

To do this, you'll need to write custom transformers where the `.transform` method carries out the `.predict` implementation.

## Data Types

What types of data can be fed in as the `X` and `y` arguments to transformers and estimators?  There are two considerations:
1. If you are writing your own transformers and estimators, you can make them take whatever file data format you like, as long as output and input types agree for sequential elements in a chain.  Scikit-learn will happily pass these objects through pipelines, feature union, grid searches, etc.
2. If you want to interact with transforms and estimators provided by Scikit-learn, you need to provide data in a compatible type.  Scikit-learn, like much of the Python ecosystem, uses duck typing.  This means that instead of testing that the input is a particular type, it tests that the input has particular capabilities.  For most elements, Scikit-learn will require that the inputs behave like Numpy arrays.  If you're not sure if a particular input will work, just try it!

## Validating Your Implementations

Once you've implemented a custom estimator/transformer, you can validate that it correctly works using `sklearn.utils.estimator_checks.check_estimator` [docs](http://scikit-learn.org/stable/modules/generated/sklearn.utils.estimator_checks.check_estimator.html). Here's an example, validating our above implementations.

**Warning**: This is a bit of code inside of sklearn that's still under active development and discussion. It should help you get the basics down for implementing sklearn's interface, but you might run into some weird issues, or this validation might be "too much" for the pipeline you're trying to build or even not general enough for something that might be compatible. You might actually have something that's compatible with `GridSearchCV` for example, and would still fail validation here (e.g. input validation below).

For example, let's try validating the transformer we wrote:

In [ ]:
from sklearn.utils import estimator_checks as ec
try:
    ec.check_estimator(CenteringTransformer)
except AssertionError as e:
    print e

As you can see, we still have an error around input validation. This exception is expected... 
http://scikit-learn.org/stable/developers/contributing.html#input-validation
And is even something worth worrying about during development. Here's the validation scikit-learn is looking for:

In [ ]:
from sklearn.utils import check_array

def validate_args(arg):
    check_array(arg)
    return np.asarray(arg)

class ValidatingCenteringTransformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    """
    Centers the features about 0
    """
    def fit(self, X, y=None):
        v_X = validate_args(X) 
        v_y = np.asarray(y) if y is not None else None
        self.means = v_X.mean(axis=0)
        return self
    def transform(self, X):
        v_X = validate_args(X)
        return v_X - self.means


ec.check_estimator(ValidatingCenteringTransformer)

And this time, we pass!

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*